In [1]:
import numpy as np
import pandas as pd
from math import floor
from scipy.stats import norm

In [2]:
S_0 = 37
K = 40
T = 0.75
Sigma = 0.28
r = 0.04
q = 0.02

tau_final = T * (Sigma ** 2) / 2
x_compute = np.log(S_0 / K)
x_left = x_compute + (r - q - Sigma ** 2 / 2) * T - 3 * Sigma * np.sqrt(T)
x_right = x_compute + (r - q - Sigma ** 2 / 2) * T + 3 * Sigma * np.sqrt(T)

a = (r - q) / Sigma ** 2 - 1 / 2
b = (a + 1) ** 2 + 2 * q / Sigma ** 2

def f(x):
    return K * np.exp(a * x) * max(1 - np.exp(x), 0)

def g_left(tau):
    return K * np.exp(a * x_left + b * tau) * (1-np.exp(x_left))

def g_right(tau):
    return 0

In [41]:
def backward_euler(M):
    alpha = 5
    delta_T = tau_final / M
    N = floor((x_right - x_left) / np.sqrt(delta_T / alpha))
    delta_x = (x_right - x_left) / N
    alpha = delta_T / delta_x ** 2
    grid = np.zeros((M+1, N+1), dtype=float)
    for i in range(M + 1):
        t_i = delta_T * i
        grid[i, 0] = g_left(t_i)
        grid[i, N] = g_right(t_i)

    for i in range(N + 1):
        x_i = x_left + i * delta_x
        grid[0, i] = f(x_i)

    bound = grid.copy()

    main_diag_element = 1 + 2 * alpha
    off_diag_element = -alpha

    A = np.eye(N-1) * main_diag_element
    A += np.eye(N-1, k=1) * off_diag_element
    A += np.eye(N-1, k=-1) * off_diag_element

    A_inv = np.linalg.inv(A)

    for m in range(1, M+1):
        b = np.zeros(N-1)
        b[0] = alpha * bound[m, 0]
        b[N-2] = alpha * bound[m, N]
        grid[m, 1:-1] = (A_inv @ (grid[m-1, 1:-1] + b))

    return grid

def crank_nicolson(M):
    alpha = 0.45
    delta_T = tau_final / M
    N = floor((x_right - x_left) / np.sqrt(delta_T / alpha))
    delta_x = (x_right - x_left) / N
    alpha = delta_T / delta_x ** 2
    grid = np.zeros((M+1, N+1), dtype=float)
    for i in range(M + 1):
        t_i = delta_T * i
        grid[i, 0] = g_left(t_i)
        grid[i, N] = g_right(t_i)

    for i in range(N + 1):
        x_i = x_left + i * delta_x
        grid[0, i] = f(x_i)

    bound = grid.copy()

    main_diag_element = 1 + alpha
    off_diag_element = -alpha / 2

    A = np.eye(N-1) * main_diag_element
    A += np.eye(N-1, k=1) * off_diag_element
    A += np.eye(N-1, k=-1) * off_diag_element

    A_inv = np.linalg.inv(A)
    B = 2 * np.eye(N-1) - A

    for m in range(1, M+1):
        b = B @ grid[m-1, 1:-1]
        b[0] += alpha / 2 * (bound[m-1, 0] + bound[m, 0])
        b[N-2] += alpha / 2 * (bound[m-1, N] + bound[m, N])
        grid[m, 1:-1] = A_inv @ b
    
    return grid


def crank_nicolson(M):
    omega = 1.2  # Relaxation parameter
    tol = 1e-6  # Tolerance for SOR convergence

    alpha = 5
    delta_T = tau_final / M
    N = floor((x_right - x_left) / np.sqrt(delta_T / alpha))
    delta_x = (x_right - x_left) / N
    alpha = delta_T / delta_x ** 2
    grid = np.zeros((M+1, N+1), dtype=float)

    for i in range(M + 1):
        t_i = delta_T * i
        grid[i, 0] = g_left(t_i)
        grid[i, N] = g_right(t_i)

    for i in range(N + 1):
        x_i = x_left + i * delta_x
        grid[0, i] = max(K - S_0 * np.exp(x_i), 0)  # Initialize with early exercise premium

    for m in range(1, M + 1):
        x_prev = grid[m-1, :].copy()
        x_new = x_prev.copy()
        diff = tol + 1
        while diff > tol:
            for j in range(1, N):
                x_new[j] = max(K - S_0 * np.exp(x_left + j * delta_x), (1 - omega) * x_new[j] + omega * (alpha / 2 * (x_new[j - 1] + x_prev[j + 1]) + x_prev[j]) / (1 + alpha))
            diff = np.linalg.norm(x_new - x_prev)
            x_prev = x_new.copy()
        grid[m, 1:-1] = x_new[1:-1]

    return grid

def bsm_solver(S):
    d1 = (np.log(S / K) + (r - q + 0.5 * Sigma**2) * T) / (Sigma * np.sqrt(T))
    d2 = d1 - Sigma * np.sqrt(T)

    return K * np.exp(-r * T) * norm.cdf(-d2) - S * np.exp(-q * T) * norm.cdf(-d1)

output1 = np.round(backward_euler(4), 6)
# pd.DataFrame(output1).to_csv('./grid4.csv')
output2 = np.round(crank_nicolson(4), 6)
# pd.DataFrame(output2).to_csv('./grid4_2.csv')

In [42]:
output1

array([[3.1457141e+01, 3.0205423e+01, 2.8937678e+01, 2.7653057e+01,
        2.6350687e+01, 2.5029668e+01, 2.3689072e+01, 2.2327944e+01,
        2.0945299e+01, 1.9540122e+01, 1.8111367e+01, 1.6657955e+01,
        1.5178775e+01, 1.3672679e+01, 1.2138485e+01, 1.0574973e+01,
        8.9808840e+00, 7.3549210e+00, 5.6957450e+00, 4.0019730e+00,
        2.2721790e+00, 5.0489100e-01, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [3.1384615e+01, 3.0128980e+01, 2.8857264e+01, 2.7568630e+01,
        2.6262236e+01, 2.4937234e+01, 2.3592787e+01, 2.2228083e+01,
        2.0842372e+01, 1.9435008e+01, 1.8005530e+01, 1.6553787e+01,
        1.5080130e+01, 1.3585721e+01, 1.2073018e+01, 1.0546543e+01,
        9.0140880e+00, 7.4886170e+00, 5.9912720e+00, 4.5560980e+00,
        3

In [ ]:
def backward_euler(M):
    alpha = 5
    delta_T = tau_final / M
    N = floor((x_right - x_left) / np.sqrt(delta_T / alpha))
    delta_x = (x_right - x_left) / N
    alpha = delta_T / delta_x ** 2
    grid = np.zeros((M+1, N+1), dtype=float)
    for i in range(M + 1):
        t_i = delta_T * i
        grid[i, 0] = g_left(t_i)
        grid[i, N] = g_right(t_i)

    for i in range(N + 1):
        x_i = x_left + i * delta_x
        grid[0, i] = f(x_i)

    bound = grid.copy()

    main_diag_element = 1 + 2 * alpha
    off_diag_element = -alpha

    A = np.eye(N-1) * main_diag_element
    A += np.eye(N-1, k=1) * off_diag_element
    A += np.eye(N-1, k=-1) * off_diag_element

    A_inv = np.linalg.inv(A)

    for m in range(1, M+1):
        b = np.zeros(N-1)
        b[0] = alpha * bound[m, 0]
        b[N-2] = alpha * bound[m, N]
        grid[m, 1:-1] = (A_inv @ (grid[m-1, 1:-1] + b))

    return grid

def crank_nicolson(M):
    alpha = 0.45
    delta_T = tau_final / M
    N = floor((x_right - x_left) / np.sqrt(delta_T / alpha))
    delta_x = (x_right - x_left) / N
    alpha = delta_T / delta_x ** 2
    grid = np.zeros((M+1, N+1), dtype=float)
    for i in range(M + 1):
        t_i = delta_T * i
        grid[i, 0] = g_left(t_i)
        grid[i, N] = g_right(t_i)

    for i in range(N + 1):
        x_i = x_left + i * delta_x
        grid[0, i] = f(x_i)

    bound = grid.copy()

    main_diag_element = 1 + alpha
    off_diag_element = -alpha / 2

    A = np.eye(N-1) * main_diag_element
    A += np.eye(N-1, k=1) * off_diag_element
    A += np.eye(N-1, k=-1) * off_diag_element

    A_inv = np.linalg.inv(A)
    B = 2 * np.eye(N-1) - A

    for m in range(1, M+1):
        b = B @ grid[m-1, 1:-1]
        b[0] += alpha / 2 * (bound[m-1, 0] + bound[m, 0])
        b[N-2] += alpha / 2 * (bound[m-1, N] + bound[m, N])
        grid[m, 1:-1] = A_inv @ b
    
    return grid


def crank_nicolson(M):
    omega = 1.2  # Relaxation parameter
    tol = 1e-6  # Tolerance for SOR convergence

    alpha = 5
    delta_T = tau_final / M
    N = floor((x_right - x_left) / np.sqrt(delta_T / alpha))
    delta_x = (x_right - x_left) / N
    alpha = delta_T / delta_x ** 2
    grid = np.zeros((M+1, N+1), dtype=float)

    for i in range(M + 1):
        t_i = delta_T * i
        grid[i, 0] = g_left(t_i)
        grid[i, N] = g_right(t_i)

    for i in range(N + 1):
        x_i = x_left + i * delta_x
        grid[0, i] = max(K - S_0 * np.exp(x_i), 0)  # Initialize with early exercise premium

    for m in range(1, M + 1):
        x_prev = grid[m-1, :].copy()
        x_new = x_prev.copy()
        diff = tol + 1
        while diff > tol:
            for j in range(1, N):
                x_new[j] = max(K - S_0 * np.exp(x_left + j * delta_x), (1 - omega) * x_new[j] + omega * (alpha / 2 * (x_new[j - 1] + x_prev[j + 1]) + x_prev[j]) / (1 + alpha))
            diff = np.linalg.norm(x_new - x_prev)
            x_prev = x_new.copy()
        grid[m, 1:-1] = x_new[1:-1]

    return grid

def bsm_solver(S):
    d1 = (np.log(S / K) + (r - q + 0.5 * Sigma**2) * T) / (Sigma * np.sqrt(T))
    d2 = d1 - Sigma * np.sqrt(T)

    return K * np.exp(-r * T) * norm.cdf(-d2) - S * np.exp(-q * T) * norm.cdf(-d1)

output1 = np.round(backward_euler(4), 6)
# pd.DataFrame(output1).to_csv('./grid4.csv')
output2 = np.round(crank_nicolson(4), 6)
# pd.DataFrame(output2).to_csv('./grid4_2.csv')
output1

In [43]:
M_list = [4, 16, 64, 256]
col = ['e', 'ep', 'e2', 'ep2', 'er', 'erp', 'd', 'g', 't']
df = pd.DataFrame(index=M_list, columns=col)
V_exact = bsm_solver(S_0)

In [44]:
e = []
e2 = []
er = []
d = []
g = []
t = []

for M in M_list:
    alpha = 5
    delta_T = tau_final / M
    N = floor((x_right - x_left) / np.sqrt(delta_T / alpha))
    i = floor((x_compute - x_left) / np.sqrt(delta_T / alpha))
    grid = crank_nicolson(M)
    x = np.linspace(x_left, x_right, N+1)
    S = K * np.exp(x)
    u = grid[-1]

    V = np.exp(- a * x - b * tau_final) * u
    V_appro = ((S[i+1]-S_0) * V[i] + (S_0-S[i]) * V[i+1]) / (S[i+1] - S[i])
    V_error = abs(V_appro - V_exact)

    u_appro = ((x[i+1]-x_compute) * u[i] + (x_compute-x[i]) * u[i+1]) / (x[i+1] - x[i])
    V_appro2 = np.exp(-a*x_compute-b*tau_final) * u_appro
    V_error2 = abs(V_appro2 - V_exact)
    
    e.append(V_error)
    e2.append(V_error2)

    N_rms = 0
    v_rms = 0
    for k in range(N+1):
        flag = (V[k] > 0.00001 * S_0)
        if flag:
            N_rms += 1
            v_appro = np.exp(-a*x[k]-b*tau_final) * u[k]
            v_exact = bsm_solver(S[k])
            v_rms += ((v_appro - v_exact) / v_exact) ** 2
    e_rms = np.sqrt(v_rms / N_rms)
    er.append(e_rms)

    delta = (V[i+1]-V[i]) / (S[i+1]-S[i])
    gamma = ((V[i+2]-V[i+1])/(S[i+2]-S[i+1]) - (V[i]-V[i-1])/(S[i]-S[i-1])) / ((S[i+2]+S[i+1])/2 - (S[i]+S[i-1])/2)
    
    u2 = grid[-2]
    V2 = np.exp(- a * x - b * (tau_final - delta_T)) * u2
    V_appro_dt = ((S[i+1]-S_0) * V2[i] + (S_0-S[i]) * V2[i+1]) / (S[i+1] - S[i])
    dt = 2 * delta_T / Sigma ** 2
    theta = (V_appro_dt - V_appro) / dt

    d.append(delta)
    g.append(gamma)
    t.append(theta)

df['e'] = e
df['ep'] = df['e'] / df['e'].shift(1)
df['e2'] = e2
df['ep2'] = df['e2'] / df['e2'].shift(1)
df['er'] = er
df['erp'] = df['er'] / df['er'].shift(1)
df['d'] = d
df['g'] = g
df['t'] = t

In [45]:
df

,e,ep,e2,ep2,er,erp,d,g,t
4,0.005934,NaN,0.006512,NaN,0.065308,NaN,-0.502949,0.016328,-2.092812
16,0.001869,0.314998,0.001320,0.202753,0.087577,1.340974,-0.556932,0.038727,-1.841472
64,0.000319,0.170750,0.000203,0.153636,0.101190,1.155441,-0.557277,0.038757,-1.813996
256,0.000035,0.108444,0.000024,0.118689,0.108221,1.069486,-0.557367,0.038764,-1.807457


In [40]:
output2 = np.round(df, 6)
output2.to_csv('./grid.csv')